In [ ]:
!pip install transformers datasets accelerate pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 844.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import pandas as pd

# Load the CSV file (replace with your actual filename)
df = pd.read_csv("/content/IndianFinancialNews.csv")

# Check column names
print(df.columns)


Index(['Unnamed: 0', 'Date', 'Title', 'Description'], dtype='object')


In [ ]:
import pandas as pd
from datasets import Dataset

# Step 1: Load your Kaggle CSV file
df = pd.read_csv("/content/IndianFinancialNews.csv")  # Replace with your actual file name

# Step 2: Drop rows with empty values in 'Description' column
df = df.dropna(subset=["Description"])

# Step 3: Reset index
df = df.reset_index(drop=True)

# Step 4: Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)



In [ ]:
from transformers import GPT2Tokenizer

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# Tokenization function
def tokenize_function(example):
    return tokenizer(example["Description"], truncation=True, padding="max_length", max_length=128)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/49290 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel, TrainingArguments

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./results",
        logging_dir="./logs",
        logging_steps=10,  # Log after every 10 steps
        per_device_train_batch_size=2,
        num_train_epochs=3,
        save_steps=500,
        save_total_limit=2,
        report_to="none" # Disable wandb logging
    ),
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False), # Use DataCollatorForLanguageModeling
    tokenizer=tokenizer,
)

# Start training
trainer.train()

/tmp/ipython-input-8-1726805453.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,5.035400
20,4.204800
30,3.659600
40,3.969600
50,4.221200
60,4.150300
70,4.011900
80,3.702900
90,4.145000
100,4.327300


In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import GPT2Tokenizer

# Load tokenizer (you can also use "./gpt2-finetuned" if it's already saved)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set pad token (important for GPT-2)
tokenizer.pad_token = tokenizer.eos_token




tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Save the fine-tuned model (you should have model already loaded)
model.save_pretrained("./gpt2-finetuned")

# Now save the tokenizer
tokenizer.save_pretrained("./gpt2-finetuned")


('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load trained model back (replace with your training output folder if different)
model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")  # Or "./gpt2-finetuned" if you've already saved tokenizer

# Set pad token if not already set
tokenizer.pad_token = tokenizer.eos_token

# Now save it again (this time it will work)
model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")


('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a starting prompt
prompt = "The biggest lesson I learned in life was"
output = generator(prompt, max_length=300, num_return_sequences=1)

# Print generated text
print(output[0]["generated_text"])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The biggest lesson I learned in life was the importance of humility, and I learned that the easiest way to be a good person is to be humble. When a person says, "I'm proud of this outfit, I'm proud of all my friends, I'm proud of my friends" that's really saying something and it's something that is very important to you because you need to learn humility.

One of the things that I learned in life was when I was in school, I would always say to my friends, "I have to be humble. I have to be humble in a way that I'll always be humble. I know my friends have a lot to learn about me and I know that if I don't learn from them, I don't know how to be successful." That's how big of a mistake I made, the other day. I just can't learn from people. There is no way to be humble. It's a huge mistake to be humble.

So there you have it. It's been a long time since I've seen such a complete reflection on what I believe in.

How did you come to be a writer?

I was on a trip a few years back. I had th

In [ ]:
import shutil
from google.colab import files

# Zip the folder
shutil.make_archive("gpt2-finetuned", 'zip', "gpt2-finetuned")

# Download the zip
files.download("gpt2-finetuned.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>